In [5]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import numpy as np

In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
file = tf.keras.utils.get_file(
    "persons.jpg",
    "https://www.saltwire.com/media/photologue/photos/cache/STJ-A01-28102019-RawlinsCrossPedestrians3_large.jpg")

In [8]:
image = file
img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)
img = tf.expand_dims(img_raw, 0)
_,height,width,_ =img.shape
img = tf.image.resize(img, (416, 416))/255

In [9]:
!mkdir -p 'saved_model_tensorrt_segunda_version_dos_anchors/my_model'
#!mkdir -p 'saved_model_tensorrt_with_NMS/my_model'

In [10]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(max_workspace_size_bytes=(1<<32))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(maximum_cached_engines=100)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir='saved_model_segunda_version_dos_anchors/my_model',
    conversion_params=conversion_params)

converter.convert()

def my_input_fn():
    for _ in range(1):
        inp1 = tf.random.normal(shape=(1, 416, 416, 3),dtype=tf.dtypes.float32)#.astype(tf.float32)
        #inp2 = np.random.normal(size=(1, 416, 416, 3)).astype(np.float32)
        yield (inp1,)
        
converter.build(input_fn=my_input_fn)
converter.save('saved_model_tensorrt_segunda_version_dos_anchors/my_model')

INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)
INFO:tensorflow:Assets written to: saved_model_tensorrt_segunda_version_dos_anchors/my_model/assets


In [11]:
saved_model_loaded = tf.saved_model.load('saved_model_tensorrt_segunda_version_dos_anchors/my_model')

In [12]:
print(list(saved_model_loaded.signatures.keys()))

['serving_default']


In [13]:
infer = saved_model_loaded.signatures["serving_default"]
print(infer.structured_outputs)

{'output_1': TensorSpec(shape=<unknown>, dtype=tf.float32, name='output_1')}


In [14]:
labeling = infer(tf.constant(img))

In [15]:
labeling

{'output_1': <tf.Tensor: shape=(1, 40, 4), dtype=float32, numpy=
 array([[[0.10107195, 0.3660918 , 0.15938213, 0.55061716],
         [0.6488148 , 0.3221298 , 0.75702345, 0.78797966],
         [0.7180405 , 0.22821233, 0.8577997 , 0.8199841 ],
         [0.51084447, 0.19385564, 0.6342795 , 0.8703406 ],
         [0.25064903, 0.36864367, 0.3876632 , 0.9355264 ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.  

In [16]:
%timeit infer(tf.constant(img))

1.86 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
import time
tiempo = []
with tf.device("GPU:0"):
    for i in range(1000):
        inicio = time.time()
        infer(tf.constant(img))
        fin = time.time()
        tiempo.append(fin-inicio)
        #print(fin-inicio)

In [28]:
1/np.mean(tiempo[1:])

510.62395346156194

In [29]:
from scipy.stats import trim_mean
print(1/trim_mean(tiempo,0.01))

514.0005156989308


In [30]:
np.mean(tiempo[1:])

0.0019583883466901962

In [26]:
1/0.00652

153.37423312883436